In [55]:
import os
import requests
import json
import re
import urllib.parse

In [56]:
with open(".env") as f:
    env = dict([line.strip().split("=") for line in f.readlines()])
    USTC_PASSPORT_USERNAME = env["USTC_PASSPORT_USERNAME"]
    USTC_PASSPORT_PASSWORD = env["USTC_PASSPORT_PASSWORD"]

In [57]:
raw_header = """
User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/118.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate, br
Upgrade-Insecure-Requests: 1
Connection: keep-alive
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: none
Sec-Fetch-User: ?1
"""

cas_login_raw_header = """
Host: passport.ustc.edu.cn
User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/118.0
Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate, br
Content-Type: application/x-www-form-urlencoded
Content-Length: 190
Origin: https://passport.ustc.edu.cn
Connection: keep-alive
Referer: https://passport.ustc.edu.cn/login?service=https%3A%2F%2Fjw.ustc.edu.cn%2Fucas-sso%2Flogin
Upgrade-Insecure-Requests: 1
Sec-Fetch-Dest: document
Sec-Fetch-Mode: navigate
Sec-Fetch-Site: same-origin
Sec-Fetch-User: ?1
"""

normal_headers = {i.split(": ")[0]: i.split(": ")[1] for i in raw_header.split("\n")[1:-1]}
cas_login_headers = {i.split(": ")[0]: i.split(": ")[1] for i in cas_login_raw_header.split("\n")[1:-1]}

def cas_login_data(lt: str) -> str:
    data = {
        "model":"uplogin.jsp",
        "CAS_LT":lt,
        "service":"https://jw.ustc.edu.cn/ucas-sso/login",
        "warn":"",
        "showCode":"",
        "qrcode": "",
        "username": USTC_PASSPORT_USERNAME,
        "password": USTC_PASSPORT_PASSWORD,
        "LT": "",
        "button": "",
    }
    cas_login_data = "&".join([f"{k}={urllib.parse.quote(v)}" for k, v in data.items()])
    return cas_login_data

In [78]:
s = requests.Session()
url = "https://jw.ustc.edu.cn/"
r = s.get(url, headers=normal_headers, allow_redirects=False)
print(s.cookies.get_dict()) # SVRNAME

url = "https://jw.ustc.edu.cn/ucas-sso/login"
r = s.get(url=url, headers=normal_headers, allow_redirects=False)
print(s.cookies.get_dict()) # SESSION

url = "https://passport.ustc.edu.cn/login?service=https%3A%2F%2Fjw.ustc.edu.cn%2Fucas-sso%2Flogin"
r = s.get(url=url, headers=normal_headers, allow_redirects=False)
token_pattern = re.compile(r"LT-[a-z0-9]+")
lt = token_pattern.findall(r.text)[0]
print(s.cookies.get_dict()) # JSESSIONID

url = "https://passport.ustc.edu.cn/login"
r = s.post(url=url, headers=cas_login_headers, data=cas_login_data(lt), allow_redirects=False)
print(s.cookies.get_dict()) # uc, TGC

url = r.headers["Location"]
r = s.get(url=url, headers=normal_headers, allow_redirects=False)
print(s.cookies.get_dict()) # no new cookies, but SESSION is now valid

url = r.headers["Location"]
r = s.get(url=url, headers=normal_headers, allow_redirects=False)
print(s.cookies.get_dict()) # no new cookies, but SESSION is now valid

{'SVRNAME': 'student6'}
{'SVRNAME': 'student6', 'SESSION': '1b98d7f9-c288-4497-9e51-75a94a00e6a9'}
{'SVRNAME': 'student6', 'SESSION': '1b98d7f9-c288-4497-9e51-75a94a00e6a9', 'JSESSIONID': '15B7406B99950FDF8E5934FA0BB96E9F'}
{'SVRNAME': 'student6', 'SESSION': '1b98d7f9-c288-4497-9e51-75a94a00e6a9', 'JSESSIONID': '15B7406B99950FDF8E5934FA0BB96E9F', 'uc': 'PB18061443', 'TGC': '37690d5c5b534aa09ab98501f9f31630'}
{'SVRNAME': 'student6', 'SESSION': '1b98d7f9-c288-4497-9e51-75a94a00e6a9', 'JSESSIONID': '15B7406B99950FDF8E5934FA0BB96E9F', 'uc': 'PB18061443', 'TGC': '37690d5c5b534aa09ab98501f9f31630'}
{'SVRNAME': 'student6', 'SESSION': '1b98d7f9-c288-4497-9e51-75a94a00e6a9', 'JSESSIONID': '15B7406B99950FDF8E5934FA0BB96E9F', 'uc': 'PB18061443', 'TGC': '37690d5c5b534aa09ab98501f9f31630'}


In [50]:
url = "https://catalog.ustc.edu.cn/get_token"
raw_header = """
Host: catalog.ustc.edu.cn
User-Agent: Mozilla/5.0 (X11; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/118.0
Accept: application/json, text/plain, */*
Accept-Language: en-US,en;q=0.5
Accept-Encoding: gzip, deflate
Connection: keep-alive
Referer: http://catalog.ustc.edu.cn/program
"""
headers = {i.split(": ")[0]: i.split(": ")[1] for i in raw_header.split("\n")[1:-1]}
s = requests.Session()
r = s.get(url=url, headers=headers, allow_redirects=False)
token = r.json()["access_token"]

print(token)

NWQyYzIyZWEyZTY5MDg0ODA0YWQxZjQ5ZGNmMjQwODA2NTFkMTgwZDlmYTlmNWU4NzBiYzgwZTg4ZTJmNDA1OQ


In [84]:
id_list = [
    149105,148662,149110,148660,149109,149118,149119
]

url = "https://jw.ustc.edu.cn/ws/schedule-table/datum"

data = {
    "lessonIds": id_list,
}

headers = normal_headers | {
    "Accept": "*/*",
    "Content-Type": "application/json;charset=UTF-8",
}

r = s.post(url=url, headers=headers, json=data, allow_redirects=False)

print(r.json())

{'result': {'lessonList': [{'id': 148660, 'code': '001108.01', 'name': '21信息与计算科学', 'courseName': '数学实验', 'courseTypeName': '理论课', 'teacherAssignmentList': [{'role': 'MAJOR', 'code': 'T2215', 'teacherId': 6419, 'personId': 193675, 'name': '翟晓雅', 'age': 29, 'titleName': '特任副研究员', 'title': {'nameZh': '博士后研究员', 'nameEn': None, 'id': 21, 'code': '70', 'enabled': True, 'bizTypeAssocs': [1, 2], 'specialistPositionLevelAssoc': 3, 'bizTypeIds': [1, 2], 'transient': False, 'name': '博士后研究员'}, 'period': 40, 'teacherLessonType': {'id': 1, 'nameZh': '主讲', 'nameEn': None, 'code': '01', 'role': 'MAJOR', 'enabled': True}, 'contactInfo': {'email': None, 'telephone': None, 'mobile': None, 'address': None, 'postcode': None, 'qq': None, 'wechat': None}}], 'requiredPeriodInfo': {'total': 40.0, 'weeks': 10, 'theory': 40.0, 'theoryUnit': None, 'requireTheory': None, 'practice': 0.0, 'practiceUnit': None, 'requirePractice': None, 'test': None, 'testUnit': None, 'requireTest': None, 'experiment': 0.0, 'experim